# Reproducing zero-shot results of ESM
- mutagenesis: BLAT_ECOLX_Ranganathan2015
- model: ESM-1v (seeds 1 to 5)

## Download and load data

In [1]:
wildtype = "HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPAAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW"
offset = 24

In [3]:
import os
import urllib.request
import csv

if not os.path.exists("data/raw_df.csv"):
    print("Downloading raw_df.csv...")
    urllib.request.urlretrieve("https://dl.fbaipublicfiles.com/fair-esm/examples/variant-prediction/data/raw_df.csv", "data/raw_df.csv")

mutants = []
target_values = []

with open("data/raw_df.csv", "r") as fd:
    reader = csv.DictReader(fd)
    for row in reader:
        if row["protein_name"] == "BLAT_ECOLX_Ranganathan2015":
            mutants.append([])
            for mutation in row["mutant"].split(":"):
                pos = int(mutation[1:-1])
                amino_acid = mutation[-1]
                mutants[-1].append((pos - offset, amino_acid))
            target_values.append(float(row["gt"]))

print(f"Loaded {len(mutants)} mutants")

Loaded 4788 mutants


## Computing zero-shot scores

- Load foundation model
- Compute zero-shot score using masked marginal strategy (strategy (a), Eq. (1))

In [8]:
import esm
import torch
from tqdm.auto import tqdm

try:
    scores = torch.load("data/scores.pt")
except FileNotFoundError:
    ensemble = [
        esm.pretrained.esm1v_t33_650M_UR90S_1,
        esm.pretrained.esm1v_t33_650M_UR90S_2,
        esm.pretrained.esm1v_t33_650M_UR90S_3,
        esm.pretrained.esm1v_t33_650M_UR90S_4,
        esm.pretrained.esm1v_t33_650M_UR90S_5,
    ]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    scores = torch.zeros(len(mutants), len(ensemble))
    with torch.no_grad():
        for i, model_factory in enumerate(ensemble):
            model, alphabet = model_factory()
            model.to(device)
            batch_converter = alphabet.get_batch_converter()
            
            # Tokenize wildtype sequence
            _, _, wt_tokens = batch_converter([("", wildtype)])
            
            for j in tqdm(range(len(mutants)), desc=f"Model {i + 1}"):
                # Mask wildtype sequence at all mutated positions
                tokens = wt_tokens.clone()
                for (pos, _) in mutants[j]:
                    tokens[:, pos] = alphabet.mask_idx
                
                # Compute log probabilities
                log_probs = torch.log_softmax(model(tokens.to(device))["logits"], dim=-1)
                log_probs = log_probs[0].cpu()
                
                # Compute masked marginal probability
                score = torch.tensor(0.0)
                for (pos, aa) in mutants[j]:
                    score += (
                        log_probs[pos, alphabet.get_idx(aa)] -
                        log_probs[pos, alphabet.get_idx(wildtype[pos])]
                    )
                scores[j, i] = score
            del model  # Free memory
    torch.save(scores, "data/scores.pt")

Model 1:   0%|          | 0/4788 [00:00<?, ?it/s]

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1v_t33_650M_UR90S_2.pt" to /workspaces/zero-shot/.torch/hub/checkpoints/esm1v_t33_650M_UR90S_2.pt


Model 2:   0%|          | 0/4788 [00:00<?, ?it/s]

Model 3:   0%|          | 0/4788 [00:00<?, ?it/s]

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1v_t33_650M_UR90S_4.pt" to /workspaces/zero-shot/.torch/hub/checkpoints/esm1v_t33_650M_UR90S_4.pt


Model 4:   0%|          | 0/4788 [00:00<?, ?it/s]

Model 5:   0%|          | 0/4788 [00:00<?, ?it/s]

## Evaluation
- abs(spearmanr(target_values, scores))

In [9]:
from scipy.stats import spearmanr

for i in range(scores.shape[-1]):
    correlation = abs(spearmanr(target_values, scores[:, i]).correlation)
    print(f"Model {i + 1}: {correlation:.3f}")
correlation = abs(spearmanr(target_values, scores.mean(dim=-1)).correlation)
print(f"Ensemble: {correlation:.3f}")

Model 1: 0.100
Model 2: 0.099
Model 3: 0.093
Model 4: 0.086
Model 5: 0.093
Ensemble: 0.098
